# "Gaussian Processes 1 - Theory"
> "We introduce the basic theory behind Gaussian Processes"

- toc:true
- branch: master
- badges: true
- comments: true
- author: John J. Molina
- categories: [Gaussian Processes]

# References

The standard textbook on Gaussian Processes (GP) is that of Rasmussen and Williams. The book by Murphy on Machine Learning also has a nice intro to GP and how they connect with other ML methods. 
Finally, the "Matrix Cookbook" has an extensive list of identities that are helpful for the GP derivations.

- [Gaussian Processes for Machine Learning](http://www.gaussianprocess.org/gpml/). C. E. Rasmussen and C. K. I. Williams, Cambridge, the MIT Press (2006)
- Machine Learning : A Probabilistic Perspective. K. P. Murphy, Cambridge, the MIT Press (2012)
- [The Matrix Cookbook](https://www2.imm.dtu.dk/pubdb/pubs/3274-full.html). K. B. Petersen and M. S. Pedersen (2012)

# Preliminaries

Manipulating GP will require a bit of matrix algebra and the use of some not very well know identities (at least to the author).
Thus, we will start by giving (without proof) the main results needed to derive the basic GP equations.

Let $\Sigma$ be a block matrix, defined as
\begin{align}
\Sigma &= \begin{pmatrix}
A & C \\
D & B
\end{pmatrix}
\end{align}

From the sub-matrices, define $E$ and $F$ as
\begin{align}
E &= A - C B^{-1} D \\
F &= B - D A^{-1} C
\end{align}

#### Determinant

The determinant of $\Sigma$ can be written in terms of that of $A$ and $F$, or $B$ and $E$, as

\begin{align}
\det{\Sigma} &= \det{A}\cdot \det{F} = \det{B}\cdot \det{E}
\end{align}

#### Inverse

The matrix inverse of $\Sigma$ can also be expressed in block form as

\begin{align}
\Sigma^{-1} &= \begin{pmatrix}
\widetilde{A} & \widetilde{C}\\
\widetilde{D} & \widetilde{B}
\end{pmatrix}\\
&=\begin{pmatrix}
E^{-1} & - A^{-1} C F^{-1} \\
-F^{-1} D A^{-1}& F^{-1}
\end{pmatrix} \\
&= \begin{pmatrix}
A^{-1} + A^{-1} C F^{-1} D A^{-1} & -E^{-1} C B^{-1} \\
-B^{-1} D E^{-1} & B^{-1} + B^{-1} D E^{-1} C B^{-1}
\end{pmatrix}
\end{align}

#### Derivatives

When "training" our GP models, it will be useful to be able to compute derivatives of these block matrices with respect to the hyper-parameters $\Theta$.
In particular, we will need the derivatives of the matrix inverse and the log of the determinant. These are given by

\begin{align}
\frac{\partial}{\partial\theta} A^{-1} &= -A^{-1} \frac{\partial A}{\partial\theta} A^{-1}\\
\frac{\partial}{\partial\theta} \log{\left(\det{A}\right)} &= \text{tr}{\left(A^{-1}\frac{\partial A}{\partial\theta}\right)}
\end{align}

#### Symmetric Matrices

In the case that $\Sigma$ is a symmetric matrix, which is the only case we will be interested in here, $\Sigma = \Sigma^{t}$, which in turn implies that $A=A^t$, $B = B^{t}$, $D = C^{t}$, the block form of the matrix inverse (also symmetric) can be written as


\begin{align}
\Sigma^{-1} &= \begin{pmatrix}
\widetilde{A} & \widetilde{C}\\
\widetilde{C}^t & \widetilde{B}
\end{pmatrix} \\
&=\begin{pmatrix}
E^{-1} & - A^{-1} C F^{-1} \\
- F^{-1} C^{t} A^{-1} & F^{-1}
\end{pmatrix} \\
&=\begin{pmatrix}
 A^{-1} + A^{-1} C F^{-1} C^{t} A^{-1}& - E^{-1} C B^{-1} \\
 -B^{-1} C^{t} E^{-1} & B^{-1} + B^{-1} C^{t} E^{-1} C B^{-1}
\end{pmatrix}
\end{align}

with 

\begin{align}
\widetilde{A}^{-1} = E &= A - C B^{-1} C^{t} \\
\widetilde{B}^{-1} = F &= B - C^{t} A^{-1} C
\end{align}

The following form of the relations will be particularly useful for the derivations below
\begin{align}
\widetilde{C} &= -E^{-1} C B^{-1} = - \widetilde{A} C B^{-1} \\
\widetilde{B} &= B^{-1} + B^{-1} C^t E^{-1} C B^{-1} = B^{-1} + B^{-1} C^t \widetilde{A} C B^{-1} \Longrightarrow
\widetilde{B} - B^{-1} = B^{-1} C^t \widetilde{A}C B^{-1}
\end{align}

# (Multi-variate) Gaussians / GP

Now we can turn our attention to GP. As defined in Rasmussen and Williams, a GP is "a collection of random variables, any finite number of which have a joint Gaussian distribution". What does this mean?

Previously, for the Bayesian Parameter Estimation problem, we were given some data and a model (which on prior information was assumed to explain the data), and tasked with finding the distribution of the parameters that could explain the data. That is, we wanted to infer or learn the parameters from the data. However, this only works if we know the model. What happens when we don't posses this information? 

This leads us to the much trickier problem of "Non-parametric Bayesian Inference". Since we don't have a model to parametrize, we take the function values themselves to be the parameters! So it's not so much that there are no parameters, it's just that there is an infinite number of them. Instead of learning the parameters in some model, we will try to learn the function itself from the data. 

In the specific case of GP, we assume that the value of the function at each point (e.g., $x(t)$) is a random variable, and that they are all correlated, with a joint Gaussian distribution. Thus, the joint probability distribution for the $x = (x(t_1), x(t_2), \ldots x(t_n)) = (x_1, x_2, \ldots x_n)$ is given by a multi-variate Gaussian, specified by some mean $\mu$ and (symmetric) covariance matrix $\Sigma$. We express this as 

\begin{align}x\sim \mathcal{N}(\mu, \Sigma)
\end{align}

which is to be interpreted according to
\begin{align}
p(x\lvert \mu, \Sigma) &= \frac{1}{\sqrt{\det{\left(2\pi\Sigma\right)}}} \exp{\left[-\frac{1}{2}\delta x^t \Sigma^{-1}\delta x\right]}\qquad \left(\delta x= x - \mu\right)\\
\int p(x\lvert \mu, \Sigma) \,\mathrm{d}x &= 1 
\end{align}

By definition the first and second moments are given by the average and covariance matrix

\begin{align}
\langle x\rangle \equiv \int x p(x\lvert \mu, \Sigma) \,\mathrm{d}x &= \mu \\
\left\langle\delta x_i \delta x_j\right\rangle \equiv \int \delta x_i \delta x_j p(x\lvert \mu,\Sigma) \,\mathrm{d}x &= \Sigma_{ij}
\end{align}

#### Digression on Gaussian Integrals

To compute the marginal and conditional distributions we will need to manipulate the quadratic expressions appearing in the exponential. 

In particular, we will need to compute integrals of the form

\begin{align}
I(A, b, c) &= \int \exp{\left[-\frac{1}{2} x^t A x + x^t b + c\right]} \,\mathrm{d}x
\end{align}
where $A$ is a symmetric symmetric. This integral can be performed easily by completing the square, as follows

\begin{align}
-\frac{1}{2} x^t A x + x^t b &= -\frac{1}{2} x^t A x + b^t x \\
&= -\frac{1}{2}\left[x^t A x - b^t x - x^t b\right] \\
&= -\frac{1}{2}\left[x^t A^t x - b^t A^{-t} A^{t} x - x^t b\right]\\
&= -\frac{1}{2}\left[\left(x^t - b^t A^{-t}\right) A^t x - x^t b\right]\\
&= -\frac{1}{2}\left[\left(x - A^{-1} b\right)^t A^t x - x^t b\right] \\
&= -\frac{1}{2}\left[\left(x - A^{-1} b\right)^t A^t x - \left(x - A^{-1} b + A^{-1} b\right)^t b\right] \\
&= -\frac{1}{2}\left[\left(x - A^{-1} b\right)^t A^t x - \left(x - A^{-1} b\right)^t b - b^t A^{-t} b\right]\\
&= -\frac{1}{2}\left[\left(x - A^{-1} b\right)^t A x - \left(x - A^{-1} b\right)^t A A^{-1}b\right] + \frac{1}{2} b^t A^{-1} b \\
&= -\frac{1}{2}\left[\left(x - A^{-1} b\right)^t A \left(x - A^{-1} b\right)\right] + \frac{1}{2} b^t A^{-1} b
\end{align}

where we have repeatedly used the fact that a scalar is by definition symmetric, so $\alpha=\alpha^t$, $x^t y = y^t x$, $x^t A y = y^t A^t x$, and so on ($\alpha$ a scalar, $x$ and $y$ vectors, and $A$ a square matrix).

\begin{align}
I(A, b, c)&= \int \exp{\left[-\frac{1}{2}x^t A x + x^t b + c\right]\mathrm{d}x} \\
&= \exp{\left[\frac{1}{2}b^t A^{-1} b + c\right]} \underbrace{\int\exp{\left[-\frac{1}{2} \left(x - A^{-1}b\right)^t A \left(x - A^{-1} b\right)\right]} \mathrm{d}x}_{\equiv \sqrt{\det{\left(2\pi A^{-1}\right)}}}\\
&= \sqrt{\det{\left(2\pi A^{-1}\right)}} \exp{\left[\frac{1}{2} b^t A^{-1} b + c\right]} \\
&= \sqrt{\frac{(2\pi)^{n}}{\det{A}}} \exp{\left[\frac{1}{2} b^t A^{-1} b + c\right]}
\end{align}

## Marginalization

Now lets consider partitioning our set of points $x$ in two, $x_A$ and $x_B$, which could represent the (known) training data and (unknown) test data, respectively.
The joint distribution, is given exactly by the expression above, but we can rewrite it in block form to highlight the contribution of the $x_A$ and $x_B$ points

\begin{align}
p(x\lvert \mu, \Sigma) &= \frac{1}{\sqrt{(2\pi)^n \det{\Sigma}}} \exp{\left[-\frac{1}{2}\delta x^t \Sigma^{-1} \delta x\right]}\, ,\qquad
\Sigma = \begin{pmatrix}\Sigma_{AA} & \Sigma_{AB} \\ \Sigma_{AB}^t & \Sigma_{BB}\end{pmatrix}\\
&= \frac{1}{\sqrt{(2\pi)^n\det{\Sigma}}} \exp{\left[-\frac{1}{2}\begin{pmatrix}\delta x_A\\\delta x_B\end{pmatrix}^t \begin{pmatrix}\widetilde{\Sigma}_{AA} & \widetilde{\Sigma}_{AB}\\
\widetilde{\Sigma}_{AB}^t & \widetilde{\Sigma}_{BB}\end{pmatrix}\begin{pmatrix}\delta x_A \\ \delta x_B\end{pmatrix}\right]}
\end{align}

Where we used the properties of block matrices to rewrite the inverse of $\Sigma$ into block form. 

Given this joint distribution for $x_A$ and $x_B$, what can we say about the distribution for $x_B$, regardless of $x_A$?

By definition, we simply marginalize over $x_A$
\begin{align}
p(x_B\lvert\mu,\Sigma) &= \int p(x_A,x_B\lvert\mu,\Sigma)\mathrm{d}x_A
\end{align}

To evaluate this integral, lets rewrite the terms appearing in the exponent, trying to separate out the $x_A$ and $x_B$ contributions

\begin{align}
\delta x^t \Sigma^{-1}\delta x &= \delta x_A^t\left(\widetilde{\Sigma}_{AA}\delta x_A + \widetilde{\Sigma}_{AB}\delta x_B\right) + \delta x_B^t \left(\widetilde{\Sigma}^t_{AB}\delta x_A + \widetilde{\Sigma}_{BB}\delta x_B\right) \\
&= \bigg[\delta x_A^t \widetilde{\Sigma}_{AA}\delta x_A + 2\delta x_A^t \widetilde{\Sigma}_{AB}\delta x_B\bigg] + \delta x_B^t \widetilde{\Sigma}_{BB}\delta x_B
\end{align}

Again, using the properties of these block matrices, the determinant in the normalization factor can be conveniently written as

\begin{align}
\det{\Sigma} = \det{\Sigma_{AA}}\cdot \det{\widetilde{\Sigma}_{BB}^{-1}} = \det{\Sigma_{BB}}\cdot \det{\widetilde{\Sigma}_{AA}^{-1}}
\end{align}

Putting all this together, the marginal distribution for $x_B$ takes the form

\begin{align}
p(x_B\lvert\mu,\Sigma) &= \left(\frac{1}{\sqrt{(2\pi)^{n_A}\det{\widetilde{\Sigma}_{AA}^{-1}}}}\underbrace{\int \exp{\left[-\frac{1}{2}\delta x_A^t \widetilde{\Sigma}_{AA}\delta x_A - \delta x_A^t \widetilde{\Sigma}_{AB}\delta x_B\right]}\mathrm{d}x_A}_{I(\widetilde{\Sigma}_{AA}, -\widetilde{\Sigma}_{AB}\delta x_B, 0)}\right)\times
\left(\frac{1}{\sqrt{(2\pi)^{n_B}\det{\Sigma_{BB}}}} \exp{\left[-\frac{1}{2}\delta x^{t}_B \widetilde{\Sigma}_{BB}\delta x_B\right]}\right) \\
&= \left(\frac{1}{\sqrt{(2\pi)^{n_A}\det{\widetilde{\Sigma}_{AA}^{-1}}}} \times \sqrt{\frac{(2\pi)^{n_A}}{\det\widetilde{\Sigma}_{AA}}} \exp{\left[\frac{1}{2}\left(\widetilde{\Sigma}_{AB}\delta x_B\right)^t\widetilde{\Sigma}_{AA}^{-1}\left(\widetilde{\Sigma}_{AB}\delta x_B\right)\right]}\right)\times
\left(\frac{1}{\sqrt{(2\pi)^{n_B}\det{\Sigma_{BB}}}} \exp{\left[-\frac{1}{2}\delta x^{t}_B \widetilde{\Sigma}_{BB}\delta x_B\right]}\right) \\
&=\frac{1}{\sqrt{(2\pi)^{n_B}\det{\Sigma_{BB}}}} \exp{\left[-\frac{1}{2}\delta x^t_B\left(\widetilde{\Sigma}_{BB}- \widetilde{\Sigma}_{AB}^t \widetilde{\Sigma}_{AA}^{-1} \widetilde{\Sigma}_{AB}\right)\delta x_B\right]}
\end{align}

This is almost in the form of a multi-variate Gaussian. We can further simplify it by using the properties of block matrices listed above, since

\begin{align}
\widetilde{\Sigma}_{AB}^t \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB} &= \widetilde{C}^t \widetilde{A}^{-1} \widetilde{C} \\
&=\left(-B^{-1} C^{t} \widetilde{A}\right) \widetilde{A}^{-1}\left(-\widetilde{A} C B^{-1}\right) \\
&= B^{-1}C^t \widetilde{A}C B^{-1}\\
&\equiv \widetilde{B} - B^{-1} \\
&= \widetilde{\Sigma}_{BB} - \Sigma_{BB}^{-1}
\end{align}

where we have used the fact that $\widetilde{C}= -\widetilde{A}C B^{-1}$.

We thus arrive at the result that the marginal distribution for $x_B$ is also Gaussian, with average $\mu_B$ and covariance matrix $\Sigma_{BB}$. 

We can simply read off the distribution for $x_B$ from the original joint distribution!
\begin{align}
p(x_B\lvert\mu,\Sigma) = p(x_B\lvert \mu_B, \Sigma_{BB}) &= \frac{1}{\sqrt{(2\pi)^{n_B}\det{\Sigma_{BB}}}} \exp{\left[-\frac{1}{2}\delta x^{t}_B \Sigma^{-1}_{BB}\delta x_B\right]} \\
x_B &\sim \mathcal{N}(\mu_B, \Sigma_{BB})
\end{align}

This is the meaning of the quoted text which says that "any finite number of which have a joint Gaussian distribution".

## Conditioning

A more useful result comes from considering the conditional distribution. Say we have already measured $x_B$, this would be our training data set, what can we say about the function values $x_A$ at other points? 

From Bayes' theorem, this conditional distribution is simply given by

\begin{align}
p(x_A\lvert x_B, \mu, \Sigma) &= \frac{p(x_A, x_B\lvert \mu, \Sigma)}{p(x_B\lvert \mu, \Sigma)}
\end{align}

After some simple manipulations, we will see that this distribution again has the form of a multi-variate Gaussian, although the average and covariance matrices will be a bit more complicated.

To start, let us rewrite the exponent appearing in the numerator, in order to cancel out the exponent in the denominator.
\begin{align}
\delta x^t \Sigma^{-1}\delta x &= \delta x_A^t\left(\widetilde{\Sigma}_{AA}\delta x_A+ \widetilde{\Sigma}_{AB}\delta x_B\right) + \delta x_B^t \left(\widetilde{\Sigma}^t_{AB}\delta x_A + \widetilde{\Sigma}_{BB}\delta x_B\right) \\
&= \bigg[\delta x_A^t \widetilde{\Sigma}_{AA}\delta x_A + 2\delta x_A^t \widetilde{\Sigma}_{AB}\delta x_B\bigg] + \delta x_B^t \widetilde{\Sigma}_{BB}\delta x_B\\
&= \bigg[\delta x_A^t \widetilde{\Sigma}_{AA}\left(\delta x_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right) + \delta x_B^t \widetilde{\Sigma}_{AB}^t \delta x_A\bigg] + \delta x_B^t \widetilde{\Sigma}_{BB}\delta x_B\\
&= \bigg[\delta x_A^t \widetilde{\Sigma}_{AA}\left(\delta x_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right) + \delta x_B^t \widetilde{\Sigma}_{AB}^t\widetilde{\Sigma}_{AA}^{-t}\widetilde{\Sigma}_{AA}^t \left(\delta x_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B - \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right)\bigg] + \delta x_B^t \widetilde{\Sigma}_{BB}\delta x_B \\
&= \bigg[\left(\delta x_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right)^t \widetilde{\Sigma}_{AA}\left(\delta x_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right)\bigg] -
\delta x_B^t \underbrace{\bigg[\widetilde{\Sigma}_{AB}^t\widetilde{\Sigma}_{AA}^{-t}\widetilde{\Sigma}_{AB} - \widetilde{\Sigma}_{BB}\bigg]}_{-\Sigma_{BB}^{-1}}\delta x_B
\end{align}

From which we see that the second term on the right hand side will exactly cancel the exponential in the denominator.

Now, lets consider the ratio of normalization constants

\begin{align}
\sqrt{\frac{(2\pi)^{n_B}\det{\Sigma_{BB}}}{(2\pi)^{n_A + n_B}\det{\Sigma}}} &= \sqrt{\frac{\det{\Sigma_{BB}}}{(2\pi)^{n_A}\det{\Sigma_{BB}}\times\det{\widetilde{\Sigma}_{AA}^{-1}}}} \\
&= \frac{1}{\sqrt{(2\pi)^{n_A} \det{\widetilde{\Sigma}_{AA}^{-1}}}}
\end{align}

As promised, the conditional distribution for $x_A$ (conditioned on $x_B$) is another Gaussian!

\begin{align}
p(x_A\lvert x_B, \mu, \Sigma) &= \frac{1}{\sqrt{(2\pi)^{n_A}\det{\widetilde{\Sigma}_{AA}^{-1}}}} \exp{\left[-\frac{1}{2}\left(x_A  - \mu_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right)^t \widetilde{\Sigma}_{AA}\left(x_A - \mu_A + \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B\right)\right]}
\end{align}
\begin{align}
x_A\lvert x_B &\sim \mathcal{N}\left(\mu_A - \widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB}\delta x_B, \widetilde{\Sigma}_{AA}^{-1}\right)
\end{align}

Since it's more convenient to express all quantities in terms of the block matrices of $\Sigma$, we can rewrite the average and covariance using the following relationships

\begin{align}
\widetilde{\Sigma}_{AA}^{-1} &= \Sigma_{AA} - \Sigma_{AB} \Sigma_{BB}^{-1} \Sigma_{AB}^t
\end{align}

\begin{align}
\widetilde{\Sigma}_{AA}^{-1}\widetilde{\Sigma}_{AB} &\equiv \widetilde{A}^{-1} \widetilde{C}\\
&= -\widetilde{A}^{-1}\widetilde{A} C B^{-1} \\
&= - C B^{-1} \\
&= -\Sigma_{AB}\Sigma_{BB}^{-1}
\end{align}

From which we obtain the equivalent expression

\begin{align}
x_A\lvert x_B \sim \mathcal{N}\left(\mu_A + \Sigma_{AB}\Sigma_{BB}^{-1}\delta x_B, \Sigma_{AA}-\Sigma_{AB}\Sigma_{BB}^{-1}\Sigma^{t}_{AB}\right)
\end{align}

This is it! This is (almost) everything we need to do some Machine Learning with GP.

## Linear Combinations

One of the benefits of using GP lies in their linearity.
If $x$ and $y$ are two GP, then any linear combination of them is also a GP.

In particular, 
\begin{align}
x&\sim \mathcal{N}(\mu_x, \Sigma_x) \\
y&\sim \mathcal{N}(\mu_y, \Sigma_y) \\
A x + B y + c &\sim \mathcal{N}(A\mu_x + B\mu_y + c, A\Sigma_x A^t + B\Sigma_y B^t)
\end{align}

Unfortunately, products of GP do not result in GP...

# A (simple) implementation

For improved numerical stability and computational cost, it is recommended not to compute the matrix inverses appearing in the expressions for the averages and covariances of the conditional distribution. A better approach, which is still quite expensive, is to use the Cholesky decomposition.

If $A$ be a positive deffinite matrix (i.e., a covariance matrix $\Sigma$), $A$ can be written as the product of a lower-triangular matrix $L$ and its transpose
\begin{align}
A&= L L^t
\end{align}
such that expression of the form $A^{-1} b = x$, for known $A$ and $b$ can be computed as
\begin{align}
(L L^t)^{-1} b &= x\\
L^{-t} L^{-1} b &= x\\
L^t \backslash \left(L \backslash b\right) &= x
\end{align}

Where we have adopted the backslash notation used by Rasmussen and Williams
\begin{align}
A x&= b\\
x &= A^{-1} b\\
x &\equiv A\backslash b
\end{align}
where it is assumed that we know $A$, but not necessarily $A^{-1}$, and $b$. This notation is useful to emphasis the fact that we don't want to calculate $A^{-1}$ explicitly, we just need its product with some vector $b$ (i.e., to solve for $x$). 

Using this Cholesky decomposition, a sandwich product of the form $b^{t} A^{-1} c$ would be expressed as
\begin{align}
b^t A^{-1}c &= b^{t}L^{-t} L^{-1} c \\
&= (L^{-1} b)^t (L^{-1} c)\\
&= w^t v
\end{align}
with $v = L\backslash c$ and $w = L\backslash b$.

In the same way, we can evaluate more complicated expression, such as $A = C^t B^{-1} C$, without directly computing $B^{-1}$.
Let $C = (c_1, c_2, \ldots, c_n)$, where $c_i$ are the column-vector components of $C$. We have

\begin{align}
A &= (c_1, c_2, \ldots, c_n)^t B^{-1} (c_1, c_2, \ldots c_n) \\
&= \begin{pmatrix}c_1\\ c_2\\ \vdots\\ c_n\end{pmatrix}
\begin{pmatrix}
B^{-1} c_1 & B^{-1} c_2 &\ldots & B^{-1} c_n
\end{pmatrix}\\
&=\begin{pmatrix}
c_1^t B^{-1}c_1 & \ldots & c_1^t B^{-1} c_n \\
\vdots & \ddots & \vdots \\
c_n^t B^{-1}c_1 & \ldots & c_n^t B^{-1} c_n
\end{pmatrix}
\end{align}

where each term is computed using the expression derived above
\begin{align}
(A)_{ij} &= c_i^t B^{-1} c_j \equiv (L\backslash c_i)^t (L\backslash c_j)
\end{align}
with $L$ now the Cholesky decomposition of $B=LL^t$.

Evaluating terms like $\log{\det{A}}$ is also considerably simplified 

\begin{align}
\log\det{A} &= \log\det{LL^t} \\
&= \log\left(\det{L}\cdot \det{L^t}\right) \\
&= 2\log\det{L} = 2\sum_i\log L_{ii}
\end{align}

However, as hinted at above, this is still not the ideal way to evaluate the GP. The reason for this is the $\mathcal{O}(n^3)$ scaling of the Cholesky decomposition.
Fortunately, in recent years advanced matrix-matrix algorithms have been developed that allow for exact calculations even on millions of points!